<center>
<H1> Calling a parallel simulation code from Julia</H1>
<img src="figures/desman.jpg" width="200" alt="Galemys pyrenaicus"/>
    <H4> Marc Fuentes, INRIA </H4>
    <H5> code and notebook available at http://github.com/aitzkora/OptimizeMPI.jl
</center>    

## Rationale
 
Imagine you are doing a Ph.D and you use a big parallel software for simulating a 
physical process ; Your advisor asks to you to optimize some physical parameters. Sometimes,  it is called inverse problem whereas running your simulation is 
called direct problem. Optimization methods are numerous, and implementing state-of-the-art methods in a low-level language such as Fortran, C or C++ is not straightforward. Our idea here, is to present, some recipes (technical and numerical), to do that with our favorite language "Julia" without recoding from scratch the direct problem. Our hypotheses are the following : 

 - distributed memory paradigm for parallelism (i.e. MPI) 
 - direct problem is Fortran or C.
 - For optimization methods needing a gradient, we will implement possibly a adjoint in
 the low-level language

To sum up with a small sketch it corresponds to 
<img src="figures/rational_call.svg" width="200"/>

## technical tools

To manage to implement our optimization process, we need mainly of two ingredients
1. calling a piece of external code 
2. running julia scripts in a MPI environment 

Let's go : 

### calling Fortran or C code

- to call a external piece of code,  we will use the *Julia* statement `ccall`. As documented
[here](https://docs.julialang.org/en/v1/base/c/), the syntax is the following

```julia
ccall((:funcName, library), returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
ccall(funcPtr, returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
```

where `function_name` is the mangled name of the C function in the shared library library. If you do not know
what is mangling, take a look at [there](https://en.wikipedia.org/wiki/Name_mangling) : Roughly languages like 
Fortran (due case insensitiness) or C++ (the same function name could have several signatures), must encode
their function names when they interoperate with C.

#### Remarks 
- library is only _formally_ a string :
- you could use `"./mylib.so"`   
- but ⚠ you **could not** use `string(pwd(),"/mylib.so")` ⚠ 
- to use a library which is not in `.`, add the path to `LD_LIBRARY_PATH` before launching **Julia** (Just tested on Linux, for MacOS or Windows
- using `dlopen` and `dlsym` one could directly use the function pointer call

To start, we could do a small C example, with a function adding 2 to its argument

In [1]:
io = open("/tmp/skel.c","w")
write(io, "int addTwo(int x) { return x+2; }")
close(io)
run(`gcc -o addTwo.so --shared /tmp/skel.c`);
w = ccall((:addTwo, "./addTwo.so"), Int32, (Int32,), 12)
run(`rm addTwo.so /tmp/skel.c`)
println("w = $w")

w = 14
